In [36]:
import numpy as np
from collections import defaultdict
import time
import sys

In [37]:
# return the list of a specific value in a array
def get_keys(dict, value):
    return [k for k,v in dict.items() if v == value]

# determine whether given value in the list
def start(list, value):
    for i in list:
        if i != value:
            return False
    return True

In [38]:
class Graph:
    def __init__(self,file_name):
        self.start_time = time.process_time_ns()
        self.file_name = file_name
        self.Q = set()   # use a set to store our Q
        self.Q_max = set() # use a set to store our Q_max
        self.adj_matrix = None
        self.degree = {}  # use a dictionary to store the degree of each vertex
        self.size = None  # the number of vertex in the original Graph
        self.neighbour = defaultdict(set)    # use a dictionary to store neighbours of each vertex

    # use input txt file to form a adjacency matrix at the same time initialize the neighbour dictionary
    def get_adj_matrix(self):
        input_graph = np.loadtxt(self.file_name)
        input_graph = input_graph.astype(int)-1
        size = max(max(input_graph[:,0]),max(input_graph[:,1]))+1
        self.adj_matrix = np.zeros((size,size))
        for pair in input_graph:
            self.adj_matrix[pair[0]][pair[1]] = 1  # initialize the adj_matrix
            self.adj_matrix[pair[1]][pair[0]] = 1
            self.neighbour[pair[0]].add(pair[1])   # initalize the neighbour dictionary
            self.neighbour[pair[1]].add(pair[0])
        self.size = self.adj_matrix.shape[0]


    # count the degree of each vertex
    # after this function, we have build the degree dictionary
    def sort_degree(self):
        for i in range(self.size):
            self.degree[i] = np.count_nonzero(self.adj_matrix[i])
        self.degree = dict(sorted(self.degree.items(), key = lambda x:(x[1])))

    def get_neighbour(self, vertex):
        return self.neighbour[vertex]


    # return the ex degree of given vertex(for comparing if two vertices have the same degree)
    def ex_deg(self, vertex):
        sum = 0
        for i in self.get_neighbour(vertex):
            sum += self.degree[i]
        return sum

    # find Rmin(list) from R(list)
    def find_Rmin(self, R, current_degree):
        dict = {}
        for vertex in R:
            dict[vertex] = current_degree[vertex]
        minimal = min(dict.values())
        return get_keys(dict, minimal)
    
    # expand the Q_max
    def expand(self, R, No): # R is list  
        end_time = time.process_time_ns()            
        if (end_time - self.start_time)/1e9 > 3600:          # if running time exceeds 60 mins, end running
            print()
            print("Time runs out!")
            print("The cardinality of clique so far is: ", len(self.Q_max))
            print("The current clique is: ", self.Q_max)
            sys.exit()

        No_copy = No.copy() 
        while len(R):
            p = R[-1]  
            if len(self.Q) +  No_copy[p] +1 > len(self.Q_max):   
                self.Q.add(p)
                Rp = list(set(R).intersection(self.get_neighbour(p)))
                if len(Rp):
                    Rp, No_1 = self.number_sort(Rp)  
                    self.expand(Rp, No_1)   
                elif len(self.Q) > len(self.Q_max):
                    self.Q_max = self.Q.copy()
                    
                    print(len(self.Q_max))
                    print(self.Q_max)
                self.Q.remove(p)
            else:
                return
            R.remove(p)

    # coloring(numbering) every vertex, principle is adjacency vertices can't use same color(number)
    # get a R list by sorting vertex's color(number) ascending order
    # return the R and No
    def number_sort(self,R, No={}): # R is list
        # NUMBER
        maxno = 0
        C = defaultdict(list)
        C[0] = []
        while len(R):
            p = R[0] 
            k = 0
            while set(C[k]).intersection(self.get_neighbour(p)):
                k += 1
            if k > maxno:
                maxno = k
                C[maxno] = []          
            No[p] = k
            C[k].append(p)
            R.remove(p)          
        # SORT
        R = []
        for k in range(maxno+1):
            for j in range(len(C[k])):
                R.append(C[k][j])
        return R, No  

    # main part of this algorithm
    def MCR(self):
        i = self.size-1
        R = [i for i in range(self.size)]
        V = [None]*self.size
        degree = self.degree.copy() # copy a degree, because degree may change in this function, and we want the original degree keep unchanged
        R_min = self.find_Rmin(R, degree)
        
        while len(R_min)!=len(R):
            if len(R_min) >= 2:
                ex_degree = {}
                for item in R_min:
                    ex_degree[item] = self.ex_deg(item)
                minimal = min(ex_degree.values())
                p = get_keys(ex_degree, minimal)[-1]
            else:
                p = R_min[0]
            V[i] = p
            R.remove(p)
            i -= 1
            for j in range(len(R)):
                if R[j] in self.get_neighbour(p):
                    degree[R[j]] -= 1
            R_min = self.find_Rmin(R, degree)
        R_min, No = self.number_sort(R_min) 

        for i in range(len(R_min)):
            V[i] = R_min[i]    

        m = max(No[q] for q in R_min) +1
        mmax = len(R_min) + max(degree.values()) - m
        m += 1 
        i = len(R_min)

        while i <= mmax:
            if i > len(V):
                if start([self.degree[q] for q in R_min], len(R_min)-1):
                    self.Q_max = R_min
            No[V[i]] = m
            m += 1
            i += 1

        for i in range(mmax+1, len(V)):
            No[V[i]] = max(degree.values()) + 1
        
        self.expand(V, No) 
        print()
        print("The cardinality of maximum clique is: ",len(self.Q_max))
        print("Maximum clique is: ",self.Q_max)

## Notice: vertex in my result starts from o not 1

In [39]:
def main(file_name):
    start = time.process_time_ns()
    graph = Graph(file_name)
    graph.get_adj_matrix()
    graph.sort_degree()
    graph.MCR()
    end = time.process_time_ns()
    print("CPU running time: {} seconds".format((end-start)/1e9))

In [40]:
file_name = ["C125.9.txt","brock200_1.txt","brock200_2.txt","p_hat300_1.txt","p_hat300_3.txt"]

In [41]:
# main("p_hat300_1.txt")

In [42]:
for i in range(5):
    print(file_name[i], end = "\n")
    main(file_name[i])
    print("-----------------------------------------------------------------------------------------------------------------------------------------------")

C125.9.txt
29
{4, 6, 8, 78, 79, 81, 82, 86, 119, 24, 28, 120, 33, 103, 106, 108, 109, 110, 111, 113, 114, 51, 116, 53, 55, 56, 121, 122, 59}
30
{0, 4, 6, 70, 8, 78, 79, 82, 24, 28, 92, 33, 98, 103, 106, 43, 109, 110, 47, 48, 111, 113, 114, 116, 53, 119, 120, 121, 122, 59}
31
{0, 4, 6, 70, 8, 10, 78, 79, 82, 84, 24, 28, 92, 33, 98, 103, 106, 43, 109, 110, 47, 48, 113, 114, 116, 53, 119, 120, 121, 122, 59}
32
{0, 4, 6, 8, 10, 17, 24, 28, 33, 43, 47, 48, 53, 59, 70, 78, 79, 82, 84, 92, 98, 100, 106, 109, 110, 113, 114, 116, 119, 120, 121, 122}
33
{6, 7, 8, 10, 12, 19, 21, 24, 26, 28, 32, 33, 42, 48, 51, 53, 55, 59, 66, 78, 79, 81, 82, 84, 89, 94, 95, 103, 110, 113, 116, 119, 122}
34
{0, 1, 4, 6, 8, 10, 17, 24, 28, 33, 43, 47, 48, 53, 59, 67, 69, 70, 76, 78, 79, 82, 84, 92, 98, 100, 109, 110, 113, 114, 116, 120, 121, 122}

The cardinality of maximum clique is:  34
Maximum clique is:  {0, 1, 4, 6, 8, 10, 17, 24, 28, 33, 43, 47, 48, 53, 59, 67, 69, 70, 76, 78, 79, 82, 84, 92, 98, 100, 109, 1

SystemExit: 

/opt/homebrew/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
